# Abstractive summaries - Train DistilBART on TWEETSUMM dataset

In [1]:
from huggingface_hub import login
import pandas as pd
import numpy as np
import os, time, datetime

from datasets import Dataset, DatasetDict

from transformers import DataCollatorForSeq2Seq, AutoTokenizer, set_seed
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl

import wandb

In [2]:
!pip freeze > requirements_bart.txt

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
ds_dir = os.path.join(os.getcwd(), 'data')
try:
    HF_TOKEN =  os.environ['HF_TOKEN']
except:
    HF_TOKEN = ""

if 'google.colab' in str(get_ipython()):
    print("Running on Colab")
    from google.colab import drive, userdata
    drive.mount('/content/drive')
    HF_TOKEN = userdata.get('HF_TOKEN')
elif os.environ.get('KAGGLE_KERNEL_RUN_TYPE') != None:
    ds_dir = '/kaggle/input/bertdata2207/'
    # ds_dir="/kaggle/input/bertdata2207/"
    from kaggle_secrets import UserSecretsClient
    print("Running on Kaggle")
    # ds_dir = "/kaggle/input/tweet-data-2106-1512/"
    user_secrets = UserSecretsClient()
    HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
    WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")
    os.environ['WANDB_API_KEY'] = WANDB_API_KEY
    os.makedirs(os.path.join(os.getcwd(), "results"), exist_ok=True)


Running on Kaggle


In [4]:
set_seed(17)

In [5]:
def get_current_time():
    return datetime.datetime.now().strftime("%d%m-%H%M")

In [6]:
run_name = f"bart-abs-{get_current_time()}"

In [7]:
os.environ["WANDB_PROJECT"] = "aiml-thesis-train-test-temp"
os.environ["WANDB_WATCH"] = "all"
wandb.init(settings=wandb.Settings(start_method="thread"), id=run_name)

wandb: Currently logged in as: dawidk5 (dawidk5ul). Use `wandb login --relogin` to force relogin


In [8]:
login(token=HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load data

In [9]:
print(ds_dir)

/kaggle/input/bertdata2207/


In [10]:
checkpoint_bart = "sshleifer/distilbart-xsum-12-6"

In [11]:
train_df_temp = pd.read_csv(os.path.join(ds_dir,"dials_abs_2607_1312_train_spc.csv"), names=['conv_id','dialogue','summary'], encoding='utf-8', dtype={'conv_id':'string', 'dialogue':'string', 'summary': 'string'})
train_df_temp = train_df_temp.convert_dtypes()
train_df_temp.drop(columns=['conv_id'], inplace=True)
train_df_temp.reset_index(drop=True, inplace=True)

val_df_temp = pd.read_csv(os.path.join(ds_dir,"dials_abs_2607_1312_valid_spc.csv"), names=['conv_id','dialogue','summary'], encoding='utf-8', dtype={'conv_id':'string', 'dialogue':'string', 'summary': 'string'})
val_df_temp = val_df_temp.convert_dtypes()
val_df_temp.drop(columns=['conv_id'], inplace=True)
val_df_temp.reset_index(drop=True, inplace=True)

test_df_temp = pd.read_csv(os.path.join(ds_dir,"dials_abs_2607_1312_test_spc.csv"), names=['conv_id','dialogue','summary'], encoding='utf-8', dtype={'conv_id':'string', 'dialogue':'string', 'summary': 'string'})
test_df_temp = test_df_temp.convert_dtypes()
test_df_temp.reset_index(drop=True, inplace=True)

print(train_df_temp.dtypes)
print(train_df_temp.head())

PD_DATASETS = {'train': train_df_temp, 'validation': val_df_temp, 'test': test_df_temp}

dialogue    string[python]
summary     string[python]
dtype: object
                                            dialogue  \
0  Customer: So neither my iPhone nor my Apple Wa...   
1  Customer: @115850 hi team! i m planning to get...   
2  Customer: @AskAmex Where do I write to address...   
3  Customer: @AmazonHelp @115821 Wow, expected 4 ...   
4  Customer: @GWRHelp I'd rather you spent some t...   

                                             summary  
0  Customer enquired about his Iphone and Apple w...  
1  Customer is eager to know about the replacemen...  
2  Signed up for an AmexCard with Delta but it di...  
3  The customer have a problem. The agent is very...  
4  Customer cannot purchase a train ticket on the...  


In [12]:
tweetsumm_abs = DatasetDict(
    {
        'train': Dataset.from_pandas(train_df_temp),
        'validation': Dataset.from_pandas(val_df_temp),
        'test': Dataset.from_pandas(test_df_temp)
    }
)

In [13]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint_bart)
print(tokenizer)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

BartTokenizerFast(name_or_path='sshleifer/distilbart-xsum-12-6', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
# Source: https://huggingface.co/docs/transformers/en/tasks/summarization

def preprocess_function(examples):
    prefix = "summarize: "
    inputs = [str(prefix) + str(dial) for dial in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True) # same params as tweetsumm paper
    labels = tokenizer(text_target=examples["summary"], max_length=80, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
tokenized_tweetsumm_abs = tweetsumm_abs.map(preprocess_function, batched=True, remove_columns=['dialogue','summary'])
print(tokenized_tweetsumm_abs["train"][1])

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Map:   0%|          | 0/109 [00:00<?, ? examples/s]

{'input_ids': [0, 18581, 3916, 2072, 35, 19458, 35, 787, 1225, 4432, 1096, 20280, 165, 328, 939, 475, 1884, 7, 120, 1257, 1754, 510, 20529, 27785, 24, 924, 15, 5, 998, 24, 34, 158, 360, 5010, 21784, 6, 64, 1717, 3922, 162, 99, 16, 24, 17487, 50118, 45443, 35, 787, 2481, 3897, 2036, 166, 348, 10, 158, 7033, 5010, 714, 114, 5, 6880, 47, 829, 16, 5009, 50, 31559, 4, 37249, 10237, 50118, 44799, 35, 787, 25146, 28780, 5148, 27785, 125, 99, 114, 939, 399, 17, 27, 90, 101, 5, 1152, 8, 236, 7, 671, 24, 50118, 45443, 35, 787, 2481, 3897, 2036, 166, 1979, 75, 28, 441, 7, 3264, 5, 23312, 2886, 4, 286, 55, 335, 15, 1830, 2886, 714, 4, 17161, 352, 3753, 15, 5, 3104, 1373, 259, 35, 1205, 640, 90, 4, 876, 73, 571, 40969, 9380, 530, 4154, 510, 975, 4, 3166, 19954, 877, 110, 2969, 4, 50118, 44799, 35, 787, 25146, 28780, 5148, 2446, 27785, 125, 209, 32, 5567, 15797, 98, 473, 24, 1266, 276, 714, 3253, 13, 209, 25, 157, 50118, 45443, 35, 787, 2481, 3897, 2036, 3216, 6, 30845, 73, 5567, 15797, 32, 45, 4973

## Setup Training Evaluation

In [16]:
!pip install -U nltk

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.


In [17]:
!pip install evaluate pyrouge rouge_score bert_score meteor

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 26.8 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 72.8 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 MB 3.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
import evaluate, nltk, csv
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")
bertscore = evaluate.load("bertscore")

nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [19]:
def compute_metrics_abs(eval_pred):
    predictions, labels = eval_pred
    # Extra line added to address an overflow: https://github.com/huggingface/transformers/issues/22634
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

    rouge_scores = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    bert_scores = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
    bert_scores.pop('hashcode')
    result = {
      **{f"rouge/{k}": round(v, 4) for k,v in rouge_scores.items()},
      **{f"bertscore/bertscore-{k}": round(np.mean(v), 4) for k,v in bert_scores.items()},
      'meteor': round(meteor.compute(predictions=decoded_preds, references=decoded_labels)['meteor'], 4),
    }
   
    result["gen_len"] = np.mean(prediction_lens)
    return result


## Train and Evaluate

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_bart)

pytorch_model.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [22]:
my_batch = data_collator(tokenized_tweetsumm_abs['train'])
assert len(my_batch) == 4 # default setting for the model

In [23]:
EXPERIMENT_PARAMS = []
BASE_PARAMS = {'lr':3e-5, 'batch_size':4, 'epochs': 6}
EXPERIMENT_PARAMS.append(BASE_PARAMS)

In [24]:
LEARN_RATES = (3e-5, 3e-4, 3e-6)
BATCH_SIZES = (4, 2, 8)
EPOCHS = (6,10)

for lr in LEARN_RATES:
    for batch_size in BATCH_SIZES:
        for epoch in EPOCHS:
            if lr == BASE_PARAMS['lr'] and batch_size == BASE_PARAMS['batch_size'] and epoch == BASE_PARAMS['epochs']:
                continue
            experiment = {'lr':lr, 'batch_size':batch_size, 'epochs': epoch}
            EXPERIMENT_PARAMS.append(experiment)

In [25]:
def run_post_training(split, test_details, test_df_temp: pd.DataFrame, tokenizer, experiment, run_name_model, epoch):
    # First line added due to label error, see 
    predictions = np.where(test_details.predictions != -100, test_details.predictions, tokenizer.pad_token_id)
    preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    test_df_temp['response'] = preds
    exp_res = None
    csv_items = {**experiment, **(test_details.metrics)}
    if not exp_res:
        exp_res = {k: list() for k in csv_items.keys()}
    else:
        for k, v in csv_items.items():
            exp_res[k].append(v)

    test_metrics_df = pd.DataFrame(exp_res)
    test_df_temp.convert_dtypes()
    test_metrics_df.convert_dtypes()
    wandb.log({run_name_model: test_details.metrics})
    preds_name = f"{split}_preds_{run_name_model.replace('-','_')}_{epoch}_bart.csv"
    metrics_name =  f"{split}_metrics_{run_name_model.replace('-','_')}_{epoch}_bart.csv"
    test_df_temp.to_csv(os.path.join(os.getcwd(), 'results', preds_name), index=False, header=False, encoding='utf-8', quoting=csv.QUOTE_ALL)
    test_metrics_df.to_csv(os.path.join(os.getcwd(), 'results', metrics_name), index=False, header=True, encoding='utf-8', quoting=csv.QUOTE_ALL)
    # Using wandb documentation: https://docs.wandb.ai/guides/artifacts
    for root, dirs, files in os.walk(os.path.join(os.getcwd(), 'results')):
        for file in files:
            artifact = wandb.Artifact(name=run_name_model, type="predictions")
            artifact.add_file(local_path=os.path.join(root, file), name=file)
            wandb.log_artifact(artifact)


In [26]:
class ExtraCallback(TrainerCallback):
    def __init__(self):
        self.experiment_rows = []
        
#     def on_log(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
#         print(len(state.log_history), state.log_history)
#         self.experiment_rows.append(state.log_history[-1])
#         wandb.log({'run_name': args.run_name, **state.log_history[-1]})
        
    def on_epoch_end(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        # Save loss from state, log current epoch to wandb
        
        # 'lr': args['learning_rate'], 'batch_size': args['per_device_train_batch_size'], 'max_epochs' args['num_train_epochs']
        wandb.log({'run_name': args.run_name, **state.log_history[-1]})
#         df = pd.DataFrame(self.experiment_rows)
#         df = df.convert_dtypes()
#         df.to_csv(os.path.join('.', 'results', args['run_name'] + ".csv", header=True, index=False))
    
    def on_train_end(self, args, state, control, **kwargs):
        # Save and upload CSVs
        df = pd.DataFrame(state.log_history)
        df = df.convert_dtypes()
        df = df.groupby(['epoch'], as_index=False).mean()
        df.to_csv(os.path.join('.', 'results', args.run_name + ".csv"), header=True, index=False)
        
        
#         for split in ('train', 'validation', 'test'):
#             test_details = trainer.predict(tokenized_tweetsumm_abs[split], metric_key_prefix=split)
#             run_post_training(split, test_details, PD_DATASETS[split], tokenizer, exp, run_name_model, state.epoch)
#         if epoch in EPOCHS:
#             trainer.push_to_hub()
        

In [27]:
exp_res = None
for count, exp in enumerate(EXPERIMENT_PARAMS):
    current_time = get_current_time()
    run_name_model = f"temp-bart-abs-{current_time}-lr-{exp['lr']}-bs-{exp['batch_size']}-maxep-{exp['epochs']}"
    print("Starting experiment", count, run_name_model, "training")
    wandb.run.name = run_name_model
    wandb.run.save()

    training_args = Seq2SeqTrainingArguments(
        output_dir=os.path.join('.', run_model_name),
        eval_strategy="epoch",
        logging_strategy="epoch",
        # logging_steps=100,
        learning_rate=exp['lr'],
        per_device_train_batch_size=exp['batch_size'],
        per_device_eval_batch_size=exp['batch_size'],
        weight_decay=0.01,
        save_strategy="epoch", # "epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        num_train_epochs=exp['epochs'],
        predict_with_generate=True,
        fp16=True,
        generation_max_length=80,
        push_to_hub=False,
        report_to="none",
        run_name=run_name_model
    )
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_tweetsumm_abs["train"].select(range(0,50)),
        eval_dataset=tokenized_tweetsumm_abs["validation"].select(range(0,10)),
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics_abs,
    )
    trainer.add_callback(ExtraCallback)
    training_start = time.time()
    trainer.train()
    training_end = time.time()
    print("Finished",  run_name_model, "time it took for training:", str(datetime.timedelta(seconds=(training_end-training_start))))

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Starting experiment 0 bart-abs-1109-1147-lr-3e-05-bs-4-maxep-6 training


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge/rouge1,Rouge/rouge2,Rouge/rougel,Rouge/rougelsum,Bertscore/bertscore-precision,Bertscore/bertscore-recall,Bertscore/bertscore-f1,Meteor,Gen Len
1,4.375500,3.195378,0.268600,0.105000,0.234000,0.233200,0.892000,0.844500,0.867500,0.155800,14.900000
2,2.938100,2.833267,0.326300,0.177000,0.273500,0.272400,0.925600,0.858800,0.890800,0.203800,16.500000
3,2.126700,2.655557,0.298200,0.156300,0.250300,0.250100,0.911700,0.852000,0.880700,0.178800,16.100000
4,1.639700,2.695941,0.359900,0.173100,0.306500,0.305300,0.909500,0.865400,0.886700,0.262400,21.200000
5,1.333700,2.775159,0.354200,0.142300,0.299300,0.298700,0.908200,0.868100,0.887600,0.233900,23.700000
6,1.119600,2.827520,0.330500,0.130500,0.274000,0.274700,0.899600,0.860900,0.879700,0.216700,22.300000


1 [{'loss': 4.3755, 'grad_norm': 14.370349884033203, 'learning_rate': 2.6153846153846157e-05, 'epoch': 1.0, 'step': 13}]


...

Epoch,Training Loss,Validation Loss,Rouge/rouge1,Rouge/rouge2,Rouge/rougel,Rouge/rougelsum,Bertscore/bertscore-precision,Bertscore/bertscore-recall,Bertscore/bertscore-f1,Meteor,Gen Len
1,1.154600,2.958326,0.384000,0.170200,0.332900,0.334000,0.915600,0.872600,0.893500,0.271100,23.000000
2,0.870400,3.037460,0.323300,0.127300,0.283600,0.283700,0.905300,0.862400,0.883200,0.227800,22.400000
3,0.530400,3.362633,0.382900,0.156800,0.316500,0.314500,0.899400,0.868000,0.883300,0.291400,26.900000
4,0.363600,3.577489,0.353200,0.127700,0.273900,0.275300,0.900500,0.867000,0.883300,0.266500,26.400000
5,0.262100,3.700132,0.379300,0.127100,0.313200,0.312300,0.905800,0.871200,0.888000,0.280900,26.200000
6,0.226700,3.815955,0.367500,0.133300,0.309900,0.308700,0.905200,0.868600,0.886400,0.269000,24.400000


1 [{'loss': 1.1546, 'grad_norm': 11.703929901123047, 'learning_rate': 2.5384615384615386e-05, 'epoch': 1.0, 'step': 13}]
2 [{'loss': 1.1546, 'grad_norm': 11.703929901123047, 'learning_rate': 2.5384615384615386e-05, 'epoch': 1.0, 'step': 13}, {'eval_loss': 2.9583256244659424, 'eval_rouge/rouge1': 0.384, 'eval_rouge/rouge2': 0.1702, 'eval_rouge/rougeL': 0.3329, 'eval_rouge/rougeLsum': 0.334, 'eval_bertscore/bertscore-precision': 0.9156, 'eval_bertscore/bertscore-recall': 0.8726, 'eval_bertscore/bertscore-f1': 0.8935, 'eval_meteor': 0.2711, 'eval_gen_len': 23.0, 'eval_runtime': 2.8473, 'eval_samples_per_second': 3.512, 'eval_steps_per_second': 1.054, 'epoch': 1.0, 'step': 13}]
3 [{'loss': 1.1546, 'grad_norm': 11.703929901123047, 'learning_rate': 2.5384615384615386e-05, 'epoch': 1.0, 'step': 13}, {'eval_loss': 2.9583256244659424, 'eval_rouge/rouge1': 0.384, 'eval_rouge/rouge2': 0.1702, 'eval_rouge/rougeL': 0.3329, 'eval_rouge/rougeLsum': 0.334, 'eval_bertscore/bertscore-precision': 0.9156,

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge/rouge1,Rouge/rouge2,Rouge/rougel,Rouge/rougelsum,Bertscore/bertscore-precision,Bertscore/bertscore-recall,Bertscore/bertscore-f1,Meteor,Gen Len
1,0.458900,3.833492,0.393300,0.151300,0.324700,0.324200,0.904500,0.869900,0.886700,0.302900,24.700000
2,0.414100,3.750737,0.375500,0.122300,0.310900,0.310300,0.895300,0.870100,0.882400,0.285200,30.500000
3,0.289000,4.050595,0.323200,0.095300,0.239900,0.239400,0.889800,0.859500,0.874300,0.233900,24.900000
4,0.185900,4.168713,0.431800,0.165200,0.355300,0.353500,0.904900,0.871400,0.887700,0.324600,26.100000
5,0.120500,4.419093,0.410600,0.146300,0.336300,0.336500,0.899800,0.870900,0.885000,0.302700,27.700000
6,0.085500,4.402860,0.333000,0.095700,0.272500,0.272100,0.891200,0.862400,0.876300,0.252000,27.300000


1 [{'loss': 0.4589, 'grad_norm': 11.128762245178223, 'learning_rate': 2.54e-05, 'epoch': 1.0, 'step': 25}]
2 [{'loss': 0.4589, 'grad_norm': 11.128762245178223, 'learning_rate': 2.54e-05, 'epoch': 1.0, 'step': 25}, {'eval_loss': 3.833491802215576, 'eval_rouge/rouge1': 0.3933, 'eval_rouge/rouge2': 0.1513, 'eval_rouge/rougeL': 0.3247, 'eval_rouge/rougeLsum': 0.3242, 'eval_bertscore/bertscore-precision': 0.9045, 'eval_bertscore/bertscore-recall': 0.8699, 'eval_bertscore/bertscore-f1': 0.8867, 'eval_meteor': 0.3029, 'eval_gen_len': 24.7, 'eval_runtime': 3.2105, 'eval_samples_per_second': 3.115, 'eval_steps_per_second': 1.557, 'epoch': 1.0, 'step': 25}]
3 [{'loss': 0.4589, 'grad_norm': 11.128762245178223, 'learning_rate': 2.54e-05, 'epoch': 1.0, 'step': 25}, {'eval_loss': 3.833491802215576, 'eval_rouge/rouge1': 0.3933, 'eval_rouge/rouge2': 0.1513, 'eval_rouge/rougeL': 0.3247, 'eval_rouge/rougeLsum': 0.3242, 'eval_bertscore/bertscore-precision': 0.9045, 'eval_bertscore/bertscore-recall': 0.86

In [28]:
def log_csv_wandb(results_path, run_name_model):
    for root, dirs, files in os.walk(results_path):
        for file in files:
            artifact = wandb.Artifact(name=run_name_model, type="predictions")
            artifact.add_file(local_path=os.path.join(root, file), name=file)
            wandb.log_artifact(artifact)

In [29]:
!ls results

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


bart-abs-1109-1147-lr-3e-05-bs-4-maxep-6.csv
bart-abs-1109-1149-lr-3e-05-bs-2-maxep-6.csv
bart-abs-1109-1149-lr-3e-05-bs-4-maxep-10.csv


In [30]:
log_csv_wandb(os.path.join(os.getcwd(), 'results'), run_name_model)

In [31]:
print("Finished all training and evaluation for", run_name)
wandb.finish()

Finished all training and evaluation for bart-abs-1109-1144


epoch,▁▁▂▂▄▄▅▅▇▇███▁▁▂▂▄▄▅▅▇▇███▁▁▂▂▄▄▅▅▇▇███
eval_bertscore/bertscore-f1,▁▇▅▆▆▄█▅▅▅▇▆▆▅▃▆▆▃
eval_bertscore/bertscore-precision,▁█▅▅▅▃▆▄▃▃▄▄▄▂▁▄▃▁
eval_bertscore/bertscore-recall,▁▅▃▆▇▅█▅▇▇█▇▇▇▅██▅
eval_gen_len,▁▂▂▄▅▄▅▄▆▆▆▅▅█▅▆▇▇
eval_loss,▃▂▁▁▁▂▂▃▄▅▅▆▆▅▇▇██
eval_meteor,▁▃▂▅▄▄▆▄▇▆▆▆▇▆▄█▇▅
eval_rouge/rouge1,▁▃▂▅▅▄▆▃▆▅▆▅▆▆▃█▇▄
eval_rouge/rouge2,▂█▆█▅▄▇▄▆▄▄▄▆▃▁▇▅▁
eval_rouge/rougeL,▁▃▂▅▅▃▇▄▆▃▆▅▆▅▁█▇▃
eval_rouge/rougeLsum,▁▃▂▅▅▃▇▄▆▃▆▅▆▅▁█▇▃


In [32]:
print("Results uploaded")

Results uploaded
